In [ ]:
import cv2
import numpy as np
import picamera2
from picamera2 import Picamera2
import os
from gpiozero import PWMOutputDevice
from time import sleep

# Constants.
INPUT_WIDTH = 640
INPUT_HEIGHT = 640
SCORE_THRESHOLD = 0.3
NMS_THRESHOLD = 0.4
CONFIDENCE_THRESHOLD = 0.9

# Text parameters.
FONT_FACE = cv2.FONT_HERSHEY_SIMPLEX
FONT_SCALE = 0.7
THICKNESS = 1

# Colors.
BLACK  = (0,0,0)
BLUE   = (255,178,50)
YELLOW = (0,255,255)

def draw_label(im, label, x, y):
    """Draw text onto image at location."""
    # Get text size.
    text_size = cv2.getTextSize(label, FONT_FACE,
                                FONT_SCALE, THICKNESS)
    dim, baseline = text_size[0], text_size[1]
    # Use text size to create a BLACK rectangle.
    cv2.rectangle(im, (x,y),
                  (x + dim[0], y + dim[1] + baseline),
                  (0,0,0), cv2.FILLED);
    # Display text inside the rectangle.
    cv2.putText(im, label, (x, y + dim[1]),
                FONT_FACE, FONT_SCALE, YELLOW,
                THICKNESS, cv2.LINE_AA)

# detection
def pre_process(input_image, net):
      # Create a 4D blob from a frame.
      blob = cv2.dnn.blobFromImage(input_image, 1/255,
                                   (INPUT_WIDTH, INPUT_HEIGHT),
                                   [0,0,0], 1, crop=False)

      # Sets the input to the network.
      net.setInput(blob)
      # Run the forward pass to get output of the output layers.
      outputs = net.forward(net.getUnconnectedOutLayersNames())
      return outputs
def post_process(input_image, outputs):
      # Lists to hold respective values while unwrapping.
      class_ids = []
      confidences = []
      boxes = []
      # Rows.
      rows = outputs[0].shape[1]
      image_height, image_width = input_image.shape[:2]
      # Resizing factor.
      x_factor = image_width / INPUT_WIDTH
      y_factor =  image_height / INPUT_HEIGHT
      # Iterate through detections.
      for r in range(rows):
            row = outputs[0][0][r]
            confidence = row[4]
            # Discard bad detections and continue.
            if confidence >= CONFIDENCE_THRESHOLD:
                  classes_scores = row[5:]
                  # Get the index of max class score.
                  class_id = np.argmax(classes_scores)
                  #  Continue if the class score is above threshold.
                  if (classes_scores[class_id] > SCORE_THRESHOLD):
                        confidences.append(confidence)
                        class_ids.append(class_id)
                        cx, cy, w, h = row[0], row[1], row[2], row[3]
                        left = int((cx - w/2) * x_factor)
                        top = int((cy - h/2) * y_factor)
                        width = int(w * x_factor)
                        height = int(h * y_factor)
                        box = np.array([left, top, width, height])
                        boxes.append(box)

      indices = cv2.dnn.NMSBoxes(boxes, confidences,
                                 CONFIDENCE_THRESHOLD, NMS_THRESHOLD)
      for i in indices:
            box = boxes[i]
            left = box[0]
            top = box[1]
            width = box[2]
            height = box[3]
            # Draw bounding box.
            cv2.rectangle(input_image, (left, top),
                          (left + width, top + height), BLUE, 3*THICKNESS)
            # Class label.
            label = "{}:{:.2f}".format(classes[class_ids[i]], confidences[i])
            # sound
            option = '-s 140 -p 95 -a 200 -v ko+f5'
            os.system("espeak {} '{}'".format(option, classes_ko[class_ids[i]]))
            # vibrate
            if ((class_ids[i] == 0) or (class_ids[i] == 5) or
                (class_ids[i] == 17) or (class_ids[i] == 18) or
                (class_ids[i] == 19) or (class_ids[i] == 20) or
                (class_ids[i] == 21) or (class_ids[i] == 22) or
                (class_ids[i] == 25) or (class_ids[i] == 36) or
                (class_ids[i] == 37) or (class_ids[i] == 38) or
                (class_ids[i] == 39) or (class_ids[i] == 40) or
                (class_ids[i] == 41) or (class_ids[i] == 42) or
                (class_ids[i] == 43) or (class_ids[i] == 44) or
                (class_ids[i] == 45) or (class_ids[i] == 48)):
                  speed = 1
                  motor.value = speed
                  sleep(0.2)
                  motor.off()
            # Draw label.
            draw_label(input_image, label, left, top)
      return input_image


# main
if __name__ == '__main__':
      # Load class names.
      classesFile_ko = "best_80.names"
      classesFile = "best.names"
      classes = None
      with open(classesFile, 'rt') as f:
            classes = f.read().rstrip('\n').split('\n')
      with open(classesFile_ko, 'rt') as f:
            classes_ko = f.read().rstrip('\n').split('\n')
      # Load image.
      cv2.startWindowThread()
      picam2 = Picamera2()
      picam2.configure(picam2.create_preview_configuration(
          main={"format": 'XRGB8888', "size": (640, 640)}))
      # picam2.start_preview(picamera2.Preview.QTGL)
      picam2.start_preview()
      picam2.start()

      # Give the weight files to the model and load the network using them.
      modelWeights = "best_80.onnx"
      net = cv2.dnn.readNet(modelWeights)

      motor = PWMOutputDevice(14, active_high=True, frequency=100)
      motor.off()

      while True:
          frame= picam2.capture_array()
          # Process image.
          detections = pre_process(frame[:,:,0:3], net)
          img = post_process(frame[:,:,0:3].copy(), detections)

          t, _ = net.getPerfProfile()
          label = 'Inference time: %.2f ms' % (
              t * 1000.0 /  cv2.getTickFrequency())
          print(label)
          cv2.putText(img, label, (20, 40), FONT_FACE, FONT_SCALE,
                      (0, 0, 255), THICKNESS, cv2.LINE_AA)
          cv2.imshow('Output', img)
          cv2.waitKey(0)





